In [6]:
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import os
import gc
import joblib
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
#Load the data and make new df
train = pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")
train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [41]:
test.shape

(200000, 24)

In [8]:
def complete_col(train,test,col):
    temp = []
    for i in train[col]:
        if not i in temp:
            temp.append(i)
    for i in test[col]:
        if not i in temp:
            temp.append(i)
    return temp

In [9]:
def label(train,test,col,new_train, new_test):
    lbl_enc = preprocessing.LabelEncoder()
    lbl_enc.fit(complete_col(train,test,col))
    new_train[col] = lbl_enc.transform(train[col]).astype('int64')
    new_test[col] = lbl_enc.transform(test[col]).astype('int64')
        
def one_hot(train, test ,col, new_train, new_test):
    encoder = OneHotEncoder()
    encoder.fit([[i] for i in complete_col(train,test,col)])
    train_data = np.array([[i] for i in train[col]])
    enc = encoder.transform(train_data).toarray()
    new_train[col] = np.array(enc[:,0]).astype('int32')
    
    test_data = np.array([[i] for i in test[col]])
    enc = encoder.transform(test_data).toarray()
    new_test[col] = np.array(enc[:,0]).astype('int32')


In [10]:
def make_df_cols(train, start, num_dfs):
    encodings = [label, one_hot]
    safe_encodings = [label]
    for df in tqdm(range(num_dfs)):
        temp_df = pd.DataFrame()
        temp_df_test = pd.DataFrame()
        for col in train.columns[1:len(train.columns)-1]:
            if test[col].nunique() > 100:
                f = np.random.choice(safe_encodings)
            else:
                f = np.random.choice(encodings)
            f(train, test, col, temp_df, temp_df_test)
        temp_df.to_csv('encodings/train_col' + str(start+df))
        temp_df_test.to_csv('encodings/test_col' + str(start+df))
        temp_df = None

In [54]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding, Isomap
import category_encoders as ce
def numify(df1,df2, cols):
    labeled_train , labeled_test = pd.DataFrame(), pd.DataFrame()
    lbl_enc = preprocessing.LabelEncoder()
    for col in cols:
        test_data = df1[col].values
        train_data = df2[col].values
        lbl_enc.fit(complete_col(df1,df2,col))
        labeled_test[col] = lbl_enc.transform(test_data).astype('int32')
        labeled_train[col] = lbl_enc.transform(train_data).astype('int32')
    return labeled_test, labeled_train

def KM(train,test,cols,temp_df, temp_df_test):
    clusts = int(len(cols)/2 * np.random.random() + 4)
    test_data, train_data = numify(test, train, cols)
    km = KMeans(clusts)
    km.fit(train_data)
    temp_train = km.predict(train_data)
    temp_test  = km.predict(test_data)
    temp_df_test[cols[0] + str(temp_df_test.shape[1])] = temp_test
    temp_df['KM' + str(temp_df.shape[1])] = temp_train

def PCA_(train,test,cols,temp_df, temp_df_test):
    labeled_test, labeled_train = numify(test,train, cols)
    comps = int(len(cols)/3 * np.random.random() + 2)
    pca = PCA(n_components=comps)
    pca.fit(labeled_test.values)
    pca_train = pca.transform(labeled_train)
    pca_test = pca.transform(labeled_test)
    for i in range(comps):
        temp_df['PCA' + str(temp_df.shape[1])] = pca_train[i]
        temp_df_test['PCA' + str(temp_df.shape[1])] = pca_test[i]
        
def any_emb(f,train,test,cols,temp_df, temp_df_test):
    labeled_test, labeled_train = numify(test,train, cols)
    comps = int(min(10, len(cols)1.5 * np.random.random()))
    embedding = f
    test_np = embedding.fit_transform(labeled_test)
    train_np = embedding.transform(labeled_train)
    for i in train_np.columns:
        temp_df['any' + str(temp_df.shape[1])] = train_np[i]
        temp_df_test['any' + str(temp_df_test.shape[1])] = test_np[i]
base_embs = [ce.BaseNEncoder(), ce.BackwardDifferenceEncoder(),
 ce.BaseNEncoder(), ce.BinaryEncoder(), 
 ce.HashingEncoder(), ce.HelmertEncoder(), 
 ce.LeaveOneOutEncoder(), ce.OneHotEncoder(), 
 ce.OrdinalEncoder(), ce.SumEncoder(), ce.PolynomialEncoder(), 
 ce.WOEEncoder()]



In [55]:
def make_df_group(train, start, num_dfs):
    cols = train.columns.drop(['id','target']).values
    encodings = [KM, PCA_]
    encodings += base_embs
    
    for df in tqdm(range(num_dfs)):
        temp_df = pd.DataFrame()
        temp_df_test = pd.DataFrame()
        col_group = [cols[np.random.randint(len(cols))] for i in range(np.random.randint(3,10))]
        
        f = np.random.choice(encodings)
        if f in base_embs:
            any_emb(f, train, test, col_group, temp_df, temp_df_test)
        else:
            f(train, test, col_group, temp_df, temp_df_test)

        temp_df.to_csv('encodings/train_group' + str(start+df))
        temp_df_test.to_csv('encodings/test_group' + str(start+df))
        
        temp_df = None 
        
make_df_group(train, 0, 50)










  0%|          | 0/50 [00:00<?, ?it/s]








  2%|▏         | 1/50 [00:06<05:23,  6.60s/it]








  4%|▍         | 2/50 [00:41<16:28, 20.60s/it]

AttributeError: 'NoneType' object has no attribute 'astype'

In [ ]:
make_df(0,50)